In [0]:
import numpy as np
import pandas as pd
import datetime
import random

# Dataset

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!unzip /content/drive/My\ Drive/dot_traffic_2015.txt.gz.zip

Archive:  /content/drive/My Drive/dot_traffic_2015.txt.gz.zip
replace dot_traffic_2015.txt.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
traffic_df = pd.read_csv('/content/dot_traffic_2015.txt.gz', compression='gzip', header=0, sep=',', quotechar='"')

In [0]:
del traffic_df['functional_classification_name']
del traffic_df['date']
del traffic_df['restrictions']
del traffic_df['year_of_data']
del traffic_df['record_type']
del traffic_df['direction_of_travel_name']
del traffic_df['fips_state_code']
del traffic_df['station_id']

In [5]:
traffic_df.head(10)

,day_of_data,day_of_week,direction_of_travel,functional_classification,lane_of_travel,month_of_data,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400
0,7,3,1,3R,1,4,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6
1,26,7,7,1U,2,9,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613
2,16,3,3,1U,0,6,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892
3,26,1,5,1U,1,4,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110
4,23,7,3,4R,0,5,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7
5,25,7,5,2U,0,7,1168,781,738,425,279,395,715,1202,1486,1819,2193,2401,2691,2508,2700,2673,2746,2564,2257,2140,2060,2156,1873,1590
6,10,5,3,3U,2,9,34,43,27,48,88,269,334,476,444,355,375,434,492,499,539,542,571,559,395,351,262,217,153,61
7,27,3,3,4U,1,10,13,3,8,3,17,47,95,158,151,151,141,184,193,219,217,238,278,250,154,130,84,56,21,14
8,26,6,7,1R,2,6,115,78,31,40,42,134,268,335,392,441,600,722,652,706,751,740,784,822,703,425,192,159,150,168
9,12,3,5,2U,2,5,189,100,115,108,219,527,1212,1596,1347,1115,1114,1126,1178,1193,1364,1662,1673,1519,1148,914,808,555,468,270


In [6]:
traffic_df.shape

(7140391, 30)

In [6]:
one_hot_column = ['direction_of_travel', 'functional_classification','direction_of_travel','lane_of_travel','month_of_data','day_of_week','day_of_data']
prefix = []
for col in one_hot_column:
  prefix.append(col.replace('_',''))
data = pd.get_dummies(traffic_df,prefix=prefix, columns=one_hot_column)
data = data[:30000]
data[:5]

,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,traffic_volume_counted_after_0900_to_1000,traffic_volume_counted_after_1000_to_1100,traffic_volume_counted_after_1100_to_1200,traffic_volume_counted_after_1200_to_1300,traffic_volume_counted_after_1300_to_1400,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400,directionoftravel_0,directionoftravel_1,directionoftravel_2,directionoftravel_3,directionoftravel_4,directionoftravel_5,directionoftravel_6,directionoftravel_7,directionoftravel_8,directionoftravel_9,functionalclassification_1R,functionalclassification_1U,functionalclassification_2U,functionalclassification_3R,functionalclassification_3U,functionalclassification_4R,...,monthofdata_11,monthofdata_12,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,dayofweek_7,dayofdata_1,dayofdata_2,dayofdata_3,dayofdata_4,dayofdata_5,dayofdata_6,dayofdata_7,dayofdata_8,dayofdata_9,dayofdata_10,dayofdata_11,dayofdata_12,dayofdata_13,dayofdata_14,dayofdata_15,dayofdata_16,dayofdata_17,dayofdata_18,dayofdata_19,dayofdata_20,dayofdata_21,dayofdata_22,dayofdata_23,dayofdata_24,dayofdata_25,dayofdata_26,dayofdata_27,dayofdata_28,dayofdata_29,dayofdata_30,dayofdata_31
0,4,3,2,4,43,78,116,144,132,115,150,184,169,136,129,89,122,124,110,69,73,28,12,6,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,381,252,218,194,220,348,453,679,826,962,1158,1379,1376,1383,1453,1617,1669,1308,1068,928,885,798,650,613,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,585,408,328,364,696,1929,4228,5634,5673,4636,3925,3827,4049,3954,4077,4244,4405,4609,4361,3272,2243,2050,1453,892,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,105,73,68,66,77,113,254,367,487,668,870,996,1003,1000,1043,1011,959,851,708,559,457,297,207,110,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,6,4,2,3,1,10,17,52,64,68,82,96,99,87,87,83,61,55,35,29,21,23,9,7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [8]:
data.shape

(30000, 116)

In [0]:
cat_index = {} # Mapping of feature -> start index of feature in a record
cat_values = {} # Mapping of feature -> list of categorical values the feature can take
# build up the cat_index and cat_values dictionary
one_hot_column = ['direction_of_travel', 'functional_classification','direction_of_travel','lane_of_travel','month_of_data','day_of_week','day_of_data']
for i, header in enumerate(data.keys()):
  if i > 23: # categorical header
    feature, value = header.split('_')
    if feature not in cat_index:
      cat_index[feature] = i
      cat_values[feature] = [value]
    else:
      cat_values[feature].append(value)
def get_onehot(record, feature):
  """
  Return the portion of `record` that is the one-hot encoding
  of feature. For example, since the feature "work" is stored
  in the indices [5:12] in each record, calling `get_range(record, "work")`
  is equivalent to accessing `record[5:12]`.
  Args:
  - record: a numpy array representing one record, formatted
  the same way as a row in `data.np`
  - feature: a string, should be an element of `catcols`
  """
  start_index = cat_index[feature]
  stop_index = cat_index[feature] + len(cat_values[feature])
  return record[start_index:stop_index]
def get_categorical_value(onehot, feature):
  """
  Return the categorical value name of a feature given
  a one-hot vector representing the feature.
  Args:
  - onehot: a numpy array one-hot representation of the feature
  - feature: a string, should be an element of `catcols`
  Examples:
  >>> get_categorical_value(np.array([0., 0., 0., 0., 0., 1., 0.]), "work")
  'State-gov'
  >>> get_categorical_value(np.array([0.1, 0., 1.1, 0.2, 0., 1., 0.]), "work")
  'Private'
  """
  # TODO
  idx = np.argmax(onehot)
  return cat_values[feature][idx]
def get_feature(record, feature):
  """
  Return the categorical feature value of a record
  """
  onehot = get_onehot(record, feature)
  return get_categorical_value(onehot, feature)
def get_features(record):
  return { f: get_feature(record, f) for f in catcols }

In [10]:
get_categorical_value(np.array([0.1, 0., 1.1, 3.3, 0., 1., 0.,0,0,0,0,0,0,0,55,0,0,0]), "dayofdata")

'15'

#DataLoader

In [8]:
np.random.seed(50) # set the numpy seed for consistent split
arr = np.arange(len(data))
np.random.shuffle(arr)
np_data=data.values
shuffled_data=np_data
total_data=len(np_data)
train_data=shuffled_data[:int(total_data*0.7)].astype(np.float32)
validation_data=shuffled_data[int(total_data*0.7):int(total_data*0.85)].astype(np.float32)
test_data=shuffled_data[int(total_data*0.85):].astype(np.float32)
print("Training dataset size: {}".format(train_data.shape[0]))
print("Validation dataset size: {}".format(validation_data.shape[0]))
print("Test dataset size: {}".format(test_data.shape[0]))

Training dataset size: 21000
Validation dataset size: 4500
Test dataset size: 4500


#Neural Network

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
def get_model_name(name, batch_size, learning_rate, epoch, momentum):
  """ Generate a name for the model consisting of all the hyperparameter values
  Args:
  config: Configuration object containing the hyperparameters
  Returns:
  path: A string with the hyperparameter name and value concatenated
  """
  if momentum:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}_momentum{4}".format(name,
            batch_size,
            learning_rate,
            epoch,momentum)
  else:
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name,
            batch_size,
            learning_rate,
            epoch)
  return path

In [0]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.fc1 = nn.Linear(116, 100) # TODO
    self.bn1 = nn.BatchNorm1d(num_features=100)
    self.act = nn.ReLU()
    self.fc2 = nn.Linear(100, 80)
    self.bn2 = nn.BatchNorm1d(num_features=80)
    self.fc3 = nn.Linear(80, 60)
    
    self.fc4 = nn.Linear(140, 80)
    self.bn4 = nn.BatchNorm1d(num_features=80)
    self.fc5 = nn.Linear(180, 100)# TODO
    self.bn5 = nn.BatchNorm1d(num_features=100)
    self.fc6 = nn.Linear(100, 116)# TODO
    
    self.sig = nn.Sigmoid()
  def forward(self, x):
    out1 = self.act(self.bn1(self.fc1(x)))
    out2 = self.act(self.bn2(self.fc2(out1)))
    out3 = self.act(self.fc3(out2))
    out4 = self.act(self.bn4(self.fc4(torch.cat((out3,out2),1))))
    out5 = self.act(self.bn5(self.fc5(torch.cat((out4,out1),1))))
    x = self.act(self.fc6(out5))
    x[:,24:] = self.sig(x[:,24:])
    return x

In [0]:
def get_accuracy(model, data_loader):
    """Return the "accuracy" of the autoencoder model across a data set
    
    Args:
       - model: the autoencoder model, an instance of nn.Module
       - data_loader: an instance of torch.utils.data.DataLoader

    Example (to illustrate how get_accuracy is intended to be called.
             depending on your variable naming this code might not work
             out of the box)

        >>> model = AutoEncoder()
        >>> vdl = torch.utils.data.DataLoader(data_valid, batch_size=256, shuffle=True)
        >>> get_accuracy(model, vdl)
    """
    total = 0
    acc = 0
    count = 0
    for j in range(24):
        for item in data_loader: # minibatches
            inp = item.detach().numpy()
            out = model(zero_out_feature(item.clone().to(device))).cpu().detach().numpy()
            for i in range(out.shape[0]): # record in minibatch
                acc += np.absolute(int(out[i][j]) - inp[i][j] )
                count += 1
    return (acc)/count

In [0]:
import matplotlib.pyplot as plt
def plot_training_curve(path):
    train_err = np.loadtxt("{}_train_acc.csv".format(path))
    val_err = np.loadtxt("{}_val_acc.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Accuracy")
    plt.plot(range(1,train_err.shape[0]+1), train_err, label="Train")
    plt.plot(range(1,val_err.shape[0]+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,train_loss.shape[0]+1), train_loss, label="Train")
    plt.plot(range(1,val_loss.shape[0]+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()
def zero_out_feature(records):
    """ Set the feature missing in records, by setting the appropriate
    columns of records to 0
    """
    col = random.randint(0, 23)
    records[:, col:24] = 0
    return records

def zero_out_random_feature(records):
    """ Set one random feature missing in records, by setting the 
    appropriate columns of records to 0
    """
    return zero_out_feature(records)
def validation_eval(model, valid_loader, criterion):
    valid_error = 0
    total_valid_loss = 0
    for data_val in valid_loader:
        recon=model(data_val.to(device))
        loss = criterion(recon[:,:24], (data_val[:,:24]).to(device))
        total_valid_loss += loss.item()
    return total_valid_loss/len(valid_loader),get_accuracy(model, valid_loader)
        
def train(model, train_loader, valid_loader, num_epochs=5, learning_rate=1e-4, batch_size=1,betas=(0.9, 0.999)):
    """ Training loop. You should update this."""
    torch.manual_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,betas=betas)
    train_err = []    
    train_loss = []  
    val_err = []  
    val_loss = []
    for epoch in range(num_epochs):
        train_error = 0
        total_train_loss = 0
        train_idx = 0
        for data in train_loader:
            data_zero = zero_out_feature(data.clone()).to(device) # zero out one categorical feature
            recon = model(data_zero)
            loss = criterion(recon[:,:24], (data[:,:24]).to(device))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_train_loss += loss.item()
            train_idx+=1
        train_err.append(get_accuracy(model,  train_loader))
        train_loss.append(total_train_loss/(train_idx+1))
        valid_loss,valid_err = validation_eval(model, valid_loader,criterion)
        val_loss.append(valid_loss)
        val_err.append(valid_err)
        train_loss.append(total_train_loss/len(train_loader))
        valid_loss,_= validation_eval(model, valid_loader,criterion)
        val_loss.append(valid_loss)
        print(("Epoch {}: Train loss: {} Train flow loss: {} |"+               
               "Validation loss: {} Valid flow loss: {}").format(
            epoch + 1, train_loss[-1],train_err[-1],val_loss[-1],val_err[-1]))
        model_path = "Encoder_Decoder_Batchsize{}_lr{}_epoch{}_betas{}".format(batch_size,learning_rate,epoch,betas)
        torch.save(model.state_dict(), model_path)
    print('Finished Training')    
    np.savetxt("{}_train_loss.csv".format(model_path), np.array(train_loss))       
    np.savetxt("{}_val_loss.csv".format(model_path), np.array(val_loss))
    return model_path


In [97]:
Encoder=AutoEncoder().to(device)
batch_size=16
train_loader =torch.utils.data.DataLoader(train_data,batch_size=batch_size) 
valid_loader =torch.utils.data.DataLoader(validation_data,batch_size=batch_size) 
test_loader =torch.utils.data.DataLoader(test_data,batch_size=batch_size) 
#Encoder.load_state_dict(torch.load('Encoder_Decoder_Batchsize16_lr0.01_epoch5_betas(0.9, 0.999)'))
path = train(Encoder, train_loader, valid_loader, num_epochs=30, learning_rate=1e-2, batch_size=batch_size)

Epoch 1: Train loss: 59935.86985724189 Train flow loss: 230.04231547619048 |Validation loss: 38283.43357989805 Valid flow loss: 234.20361111111112
Epoch 2: Train loss: 42854.662386985015 Train flow loss: 218.1519126984127 |Validation loss: 32404.680387023494 Valid flow loss: 220.83524074074074
Epoch 3: Train loss: 40781.834249712905 Train flow loss: 210.01002579365078 |Validation loss: 28235.741488842254 Valid flow loss: 211.72708333333333
Epoch 4: Train loss: 38680.80835388602 Train flow loss: 199.33608928571428 |Validation loss: 28896.306806848403 Valid flow loss: 199.78192592592592
Epoch 5: Train loss: 36615.520524547195 Train flow loss: 181.5893888888889 |Validation loss: 27757.912007216866 Valid flow loss: 185.63072222222223
Epoch 6: Train loss: 35744.83666611008 Train flow loss: 174.4555376984127 |Validation loss: 22221.39323479402 Valid flow loss: 175.30051851851852
Epoch 7: Train loss: 34485.59338230154 Train flow loss: 168.30998809523808 |Validation loss: 25527.71010421861 Val

In [0]:
!cp -r /content/Encoder_Decoder_Batchsize16_lr0\.01_epoch28_betas\(0.9,\ 0.999\) /content/drive/My\ Drive/

In [0]:
def zero_out_feature_fixed(records, col):
    """ Set the feature missing in records, by setting the appropriate
    columns of records to 0
    """
    records[:, col:24] = 0
    return records

In [112]:
val = np.array([])
Encoder.load_state_dict(torch.load('Encoder_Decoder_Batchsize16_lr0.01_epoch28_betas(0.9, 0.999)'))
for col in range(23):
  for data in test_loader:
    data_zero = zero_out_feature_fixed(data.clone(), col+1) # zero out one categorical feature
    recon = Encoder(data_zero.to(device))
    diff= np.absolute(recon.cpu().detach().numpy().astype(np.int32) - data.detach().numpy().astype(np.int32))
    val = np.hstack((val,diff[:,col+1]))
print(np.mean(val))
print(np.std(val))

141.85046376811593
532.4428779380326


In [0]:
class AutoEncoder_2(nn.Module):
  def __init__(self):
    super(AutoEncoder_2, self).__init__()
    self.fc1 = nn.Linear(116, 100) # TODO
    self.bn1 = nn.BatchNorm1d(num_features=100)
    self.act = nn.ReLU()
    self.fc2 = nn.Linear(100, 80)
    self.bn2 = nn.BatchNorm1d(num_features=80)
    self.fc5 = nn.Linear(180, 100)# TODO
    self.bn5 = nn.BatchNorm1d(num_features=100)
    self.fc6 = nn.Linear(100, 116)# TODO
    
    self.sig = nn.Sigmoid()
  def forward(self, x):
    out1 = self.act(self.bn1(self.fc1(x)))
    out2 = self.act(self.bn2(self.fc2(out1)))
    out5 = self.act(self.bn5(self.fc5(torch.cat((out2,out1),1))))
    x = self.act(self.fc6(out5))
    x[:,24:] = self.sig(x[:,24:])
    return x

In [0]:
def train_2(model, train_loader, valid_loader, num_epochs=5, learning_rate=1e-4, batch_size=1,betas=(0.9, 0.999)):
    """ Training loop. You should update this."""
    torch.manual_seed(42)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,betas=betas)
    train_err = []    
    train_loss = []  
    val_err = []  
    val_loss = []
    for epoch in range(num_epochs):
        train_error = 0
        total_train_loss = 0
        train_idx = 0
        for data in train_loader:
            data_zero = zero_out_feature(data.clone()).to(device) # zero out one categorical feature
            recon = model(data_zero)
            loss = criterion(recon[:,:24], (data[:,:24]).to(device))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            total_train_loss += loss.item()
            train_idx+=1
        train_err.append(get_accuracy(model,  train_loader))
        train_loss.append(total_train_loss/(train_idx+1))
        valid_loss,valid_err = validation_eval(model, valid_loader,criterion)
        val_loss.append(valid_loss)
        val_err.append(valid_err)
        train_loss.append(total_train_loss/len(train_loader))
        valid_loss,_= validation_eval(model, valid_loader,criterion)
        val_loss.append(valid_loss)
        print(("Epoch {}: Train loss: {} Train flow loss: {} |"+               
               "Validation loss: {} Valid flow loss: {}").format(
            epoch + 1, train_loss[-1],train_err[-1],val_loss[-1],val_err[-1]))
        model_path = "Encoder_Decoder_2_Batchsize{}_lr{}_epoch{}_betas{}".format(batch_size,learning_rate,epoch,betas)
        torch.save(model.state_dict(), model_path)
    print('Finished Training')    
    np.savetxt("{}_train_loss.csv".format(model_path), np.array(train_loss))       
    np.savetxt("{}_val_loss.csv".format(model_path), np.array(val_loss))
    return model_path


In [40]:
Encoder_2=AutoEncoder_2().to(device)
batch_size=32
train_loader =torch.utils.data.DataLoader(train_data,batch_size=batch_size) 
valid_loader =torch.utils.data.DataLoader(validation_data,batch_size=batch_size) 
test_loader =torch.utils.data.DataLoader(test_data,batch_size=batch_size) 
Encoder_2.load_state_dict(torch.load('Encoder_Decoder_2_Batchsize32_lr0.01_epoch29_betas(0.9, 0.999)'))
path = train_2(Encoder_2, train_loader, valid_loader, num_epochs=30, learning_rate=1e-2, batch_size=batch_size)

Epoch 1: Train loss: 105445.09206739916 Train flow loss: 127.52561111111112 |Validation loss: 37702.07555546321 Valid flow loss: 124.41466666666666
Epoch 2: Train loss: 106411.21996810193 Train flow loss: 126.70692658730158 |Validation loss: 39444.38995144892 Valid flow loss: 123.21080555555555
Epoch 3: Train loss: 107895.85908515268 Train flow loss: 126.85868253968253 |Validation loss: 37627.51861702128 Valid flow loss: 123.89842592592592
Epoch 4: Train loss: 101616.1810869423 Train flow loss: 126.18658333333333 |Validation loss: 40532.885932651814 Valid flow loss: 123.37869444444445
Epoch 5: Train loss: 101551.034307146 Train flow loss: 124.49466468253968 |Validation loss: 37734.546488876884 Valid flow loss: 122.73361111111112
Epoch 6: Train loss: 105231.04147715111 Train flow loss: 127.44966468253968 |Validation loss: 37773.263367132095 Valid flow loss: 125.7513425925926
Epoch 7: Train loss: 104844.2084790002 Train flow loss: 126.33702976190476 |Validation loss: 38409.70511968085 Va

In [0]:
!cp -r /content/Encoder_Decoder_2_Batchsize32_lr0\.01_epoch28_betas\(0.9,\ 0.999\) /content/drive/My\ Drive/

In [57]:
val = np.array([])
Encoder_2.load_state_dict(torch.load('Encoder_Decoder_2_Batchsize32_lr0.01_epoch28_betas(0.9, 0.999)'))

for col in range(23):
  for data in test_loader:
    data_zero = zero_out_feature_fixed(data.clone(), col+1) # zero out one categorical feature
    recon = Encoder_2(data_zero.to(device))
    est = recon.cpu().detach().numpy().astype(np.int32)
    target = data.detach().numpy().astype(np.int32)
    diff= np.absolute(est - target)
    val = np.hstack((val,diff[:,col+1]))
print(np.mean(val))
print(np.std(val))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  


流式输出内容被截断，只能显示最后 5000 行内容。
  0.17977528  0.6962963 ]
[0.05494505 1.23529412 0.34519573 0.13636364 0.23444976 0.22222222
 0.3490566  0.13435374 0.6625     0.07594937 0.43362832 0.44831591
 0.42976356 0.08854167 0.19875776 0.34954407 0.30029155 0.22505308
 0.50614754 0.27880184 0.41791045 0.09333333 0.52830189 0.20295983
 0.24038462 0.30495929 0.01630435 0.65626478 0.5177305  0.56470588
 0.02702703 0.13991323]
[0.06197655 0.59813084 1.75       0.27241379 0.37248996 0.2911726
 3.85714286 0.30491991 0.24403183 0.20359066 0.19487179 0.41133721
 0.29900332 0.19157088 0.3564719  0.31343284 0.19243986 0.33789954
 0.45885287 0.13080169 0.20159681 0.08715596 0.21928817 0.32706767
 0.45384615 0.24711316 0.24736415 0.17647059 0.44370861 0.17241379
 0.34553734 0.31550802]
[       inf 1.15       0.16513761 0.53650794 1.1025641  0.56120527
 0.41176471 0.87116564 3.13333333 0.18871252 0.27802691 1.
 0.51662971 1.11078717 0.37873754 0.56470588 0.83076923 0.22476587
 0.58356164 0.2        0.41842105 0.1